In [57]:
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import pandas as pd
from tqdm import tqdm
import atlite
from scipy import interpolate
import math

The objective of this code is to estimate generation from offshore wind farms using a virtual wind farm model. In this case, we have no best behaving power curve, as observations of offshore wind generation are non-existing at the moment. Therefore, we use a best guess turbine (provided by Charlene) and the estimated locations, as well as a hub height of 150 m (on the middle-to-low end of what was predicted a bit over 10 years ago and certainly a realistic value).

### Step 1: Load farm data

In [ ]:
df_ic_offshore = pd.read_csv('../data/ic_offshore_wind_2030.csv', index_col=0)

In [38]:
df_capacity_offshore = df_ic_offshore.rename({'lat':'y', 'lon':'x', 'ic':'Capacity'}, axis='columns')

### Step 2: Load the ERA5 data and convert it into wind speed

In [22]:
dates = pd.date_range('1940-01-01', '2023-12-31', freq='1M')

In [ ]:
path_base_wind = '../data/ERA5/ERA5_model_levels/u_v_{}_{:02d}.nc'
path_base_geoh = '../data/ERA5/ERA5_model_levels/z_ml_{}_{:02d}.nc'

path_wind_128 = '../data/ERA5/ERA5_model_levels/u_v_level128_{}_{:02d}.nc'

In [24]:
ds_wind = xr.open_mfdataset([path_base_wind.format(date.year, date.month) for date in dates])
ds_geoh = xr.open_mfdataset([path_base_geoh.format(date.year, date.month) for date in dates])

ds_wind_128 = xr.open_mfdataset([path_wind_128.format(date.year, date.month) for date in dates])

In [25]:
ds_wind_128 = ds_wind_128.assign_coords({'level':128})
ds_wind = xr.concat([ds_wind_128, ds_wind], dim='level')

### Load the surface roughness

In [ ]:
ds_roughness_2018_2023 = xr.open_dataset('.../data/ERA5/ERA5_forecast_surface_roughness_geopotential_hourly_2018_2023.nc')
ds_roughness_2018_2023 = ds_roughness_2018_2023.reduce(np.nanmean, dim='expver',keep_attrs=True)

ds_roughness_2018_2023 = ds_roughness_2018_2023.drop_vars('z')

In [ ]:
ds_roughness_1988_2017 = xr.open_mfdataset(['../data/ERA5/ERA5_forecast_surface_roughness_geopotential_hourly_{}_{}.nc'.format(yy, yy+5) for yy in np.arange(1988, 2013, 6, dtype=int)])

ds_roughness_1988_2017 = ds_roughness_1988_2017.drop_vars('z')

In [ ]:
list_files_era5_old = []
filepath_base = '../data/ERA5/ERA5_all_variables_{}_{:02d}.nc'
for year in np.arange(1940,1988, dtype=int):
    for month in np.arange(1,13, dtype=int):
        list_files_era5_old.append(filepath_base.format(year, month))

ds_1940_1987 = xr.open_mfdataset(list_files_era5_old)

ds_1940_1987 = ds_1940_1987[['fsr']]

In [29]:
ds_roughness = xr.concat([ds_1940_1987, ds_roughness_1988_2017, ds_roughness_2018_2023], dim='time')

In [30]:
da_roughness = ds_roughness['fsr']

In [31]:
ds_wind['longitude']  = ds_wind['longitude'] - 360.
ds_geoh['longitude']  = ds_geoh['longitude'] - 360.

In [32]:
z = (ds_geoh['z']/9.80665).rename({'hybrid':'level'})
u = ds_wind['u']
v = ds_wind['v']

In [33]:
z = z - z[:,0]

In [34]:
# Fix the order of dimensions and the number of levels
u = u.transpose(*(z.dims))
v = v.transpose(*(z.dims))
z = z[:,:-2]

#### Keep only the points nearest to the farms

In [35]:
def create_nearest_point_mask(dataarray, target_lats, target_lons):
    """
    Create a boolean mask where the nearest grid points to each lat and lon in the lists are True, and False otherwise.

    Parameters:
    - dataarray (xarray.DataArray): Input data array with lat/lon dimensions.
    - target_lats (list or array): List of target latitudes.
    - target_lons (list or array): List of target longitudes.

    Returns:
    - mask (xarray.DataArray): A boolean mask with True at the nearest grid points, False elsewhere.
    """
    # Ensure target latitudes and longitudes are the same length
    if len(target_lats) != len(target_lons):
        raise ValueError("The number of target latitudes and longitudes must match.")

    # Extract lat and lon arrays from the DataArray
    lats = dataarray['latitude'].values
    lons = dataarray['longitude'].values

    # Create an empty mask with the same shape as the lat/lon grid
    mask = np.full((len(lats), len(lons)), False)

    # Create 2D grids of lat/lon
    lon_grid, lat_grid = np.meshgrid(lons, lats)

    # Loop over each pair of target lat and lon
    for target_lat, target_lon in zip(target_lats, target_lons):
        # Calculate the Euclidean distance to all grid points
        distances = np.sqrt((lat_grid - target_lat)**2 + (lon_grid - target_lon)**2)

        # Find the index of the minimum distance (nearest point)
        nearest_idx = np.unravel_index(np.argmin(distances), distances.shape)

        # Set the nearest point to True in the mask
        mask[nearest_idx] = True

    # Return the mask as a DataArray with the same lat/lon coordinates as the input dataarray
    return xr.DataArray(mask, coords={'latitude': lats, 'longitude': lons}, dims=['latitude', 'longitude'])

In [39]:
u_sel = u.where(create_nearest_point_mask(u, df_capacity_offshore['y'], df_capacity_offshore['x']))
v_sel = v.where(create_nearest_point_mask(v, df_capacity_offshore['y'], df_capacity_offshore['x']))
z_sel = z.where(create_nearest_point_mask(z, df_capacity_offshore['y'], df_capacity_offshore['x']))

### Get the levels to use for the calculation at 150 meters

In [40]:
# Define the target height
target_height = 150.0

# Step 1: Calculate min and max heights for each point to identify boundary conditions
min_z = z_sel.min(dim='level').compute()
max_z = z_sel.max(dim='level').compute()

# Step 2: Create masks to handle cases where the target height is below min or above max
below_min_mask = target_height <= min_z
above_max_mask = target_height >= max_z

# Step 3: Replace NaNs in over and under levels to avoid errors during argmin/argmax
# For the level_over, replace NaNs with max level index if height is above max, else fill as needed
z_above = (z_sel - target_height).where((z_sel - target_height) >= 0, np.inf)
level_over = z_above.argmin(dim='level')

# For the level_under, replace NaNs with min level index if height is below min, else fill as needed
z_below = (z_sel - target_height).where((z_sel - target_height) <= 0, -np.inf)
level_under = z_below.argmax(dim='level')

# Step 4: Compute the levels and handle boundary cases
level_over = level_over.compute().astype(int)
level_under = level_under.compute().astype(int)

# Use the lowest level when target height is below min; highest level when above max
level_over = xr.where(below_min_mask, 0, level_over)  # If below min, take lowest level
level_under = xr.where(above_max_mask, z.level.size - 1, level_under)  # If above max, take highest level

# Ensure indices are within the bounds of the array
level_over = np.clip(level_over, 0, z.level.size - 1)
level_under = np.clip(level_under, 0, z.level.size - 1)

# Step 5: Convert indices to the correct shape for advanced indexing
# Expand the indices to match the shape of (time, lat, lon, 1) for use with np.take_along_axis
expanded_level_over = level_over.expand_dims(dim='level', axis=1)
expanded_level_under = level_under.expand_dims(dim='level', axis=1)

# Step 6: Convert to NumPy arrays for indexing
z_data = z_sel.compute().data
u_data = u_sel.compute().data
v_data = v_sel.compute().data

# Step 7: Extract above and below values using advanced indexing
z_over = np.take_along_axis(z_data, expanded_level_over, axis=1).squeeze(axis=1)
z_under = np.take_along_axis(z_data, expanded_level_under, axis=1).squeeze(axis=1)
u_over = np.take_along_axis(u_data, expanded_level_over, axis=1).squeeze(axis=1)
u_under = np.take_along_axis(u_data, expanded_level_under, axis=1).squeeze(axis=1)
v_over = np.take_along_axis(v_data, expanded_level_over, axis=1).squeeze(axis=1)
v_under = np.take_along_axis(v_data, expanded_level_under, axis=1).squeeze(axis=1)

# Step 8: Calculate distances from the target height
distance_over = z_over - target_height
distance_under = target_height - z_under

# Replace zero distances with NaNs to avoid division by zero
distance_over = np.where(distance_over != 0, distance_over, np.nan)
distance_under = np.where(distance_under != 0, distance_under, np.nan)

# Step 9: Calculate inverse distance weights
weight_over = 1.0 / distance_over
weight_under = 1.0 / distance_under

# If no valid level above or below, set the weights of the closest level to 1
weight_over = np.nan_to_num(weight_over, nan=1.0)
weight_under = np.nan_to_num(weight_under, nan=1.0)

# Step 10: Calculate 100-meter wind components using inverse distance weighting
u_150 = (weight_over * u_over + weight_under * u_under) / (weight_over + weight_under)
v_150 = (weight_over * v_over + weight_under * v_under) / (weight_over + weight_under)

# Convert the results back into xarray DataArrays with the appropriate dimensions
u_150 = xr.DataArray(u_150, coords=[z.coords['time'], z.coords['latitude'], z.coords['longitude']], dims=['time', 'latitude', 'longitude'])
v_150 = xr.DataArray(v_150, coords=[z.coords['time'], z.coords['latitude'], z.coords['longitude']], dims=['time', 'latitude', 'longitude'])


/Users/aina/anaconda3/envs/atlite/lib/python3.11/site-packages/dask/array/reductions.py:618: RuntimeWarning: All-NaN slice encountered
  return np.nanmin(x_chunk, axis=axis, keepdims=keepdims)


### Create a dataset with the variables necessary for the cutout

In [41]:
u_150.attrs['units'] = 'm s**-1'
v_150.attrs['units'] = 'm s**-1'

In [42]:
ds = xr.Dataset({'u100':u_150, 'v100':v_150, 'fsr':da_roughness}) # we assign the wind to 100 m in the variable name to avoid an error, but it is actually interpolated to 200 m. Along these lines, we will have to tell atlite that the turbine is at 100 m so that it does not vertically extrapolate the wind

### Create the cutout

In [51]:
cutout_ie = atlite.cutout.get_cutout_from_era5_data('path', ds, ['wind'])

In [52]:
cutout_ie

<Cutout "path">
 x = -11.00 ⟷ -5.00, dx = 0.25
 y = 51.00 ⟷ 56.00, dy = 0.25
 time = 1940-01-01 ⟷ 2023-12-31, dt = H
 module = era5
 prepared_features = ['wind']

### Step 3: Load the power curve data

In [53]:
path_power_curve_raw = '../../ramping/data/data_others/IEA_15MW_240_RWT.csv'
df_power_curve_raw = pd.read_csv(path_power_curve_raw)[['Wind Speed [m/s]', 'Power [kW]']]
df_power_curve_raw = df_power_curve_raw.rename({'Wind Speed [m/s]':'windspeed', 'Power [kW]':'power'}, axis=1)

In [54]:
# Sample input: wind speeds and corresponding power values
wind_speeds_input = df_power_curve_raw['windspeed'].values  # Example wind speeds
power_values_input = df_power_curve_raw['power'].values  # Corresponding power values

# Define the full wind speed range from 0 to 40 m/s with 0.01 m/s intervals
wind_speeds_interp = np.arange(0, 40.01, 0.01)

# Interpolation
# Create an interpolation function based on the input data
interp_function = interpolate.interp1d(wind_speeds_input, power_values_input, kind='cubic', fill_value=0, bounds_error=False)

# Apply the interpolation function to the new wind speed range
power_values_interp = interp_function(wind_speeds_interp)

# Ensure power is 0 before the first and after the last known wind speed
first_wind_speed = wind_speeds_input[0]
last_wind_speed = wind_speeds_input[-1]

power_values_interp[wind_speeds_interp < first_wind_speed] = 0
power_values_interp[wind_speeds_interp > last_wind_speed] = 0

### Smooth the power curve

In [55]:
# Gaussian filter function Γ(n, σ)
def gaussian_filter(n, sigma):
    return (1 / (math.sqrt(2 * np.pi) * sigma)) * np.exp(-n**2 / (2 * sigma**2))

# Function to smooth the power curve with a selected smoothing level
def smooth_power_curve(unsmoothed_curve, wind_speeds, smoothing_level):
    smoothed_curve = np.zeros_like(unsmoothed_curve)  # Initialize smoothed power curve array
    step = 0.01  # Resolution of 0.01 m/s

    for i, x in enumerate(wind_speeds):
        # Adjust σ based on wind speed and the selected smoothing level
        sigma = 0.6 + 0.2 * smoothing_level * x

        # Summing from -4σ to +4σ
        aggregate_power = 0
        normalization_factor = 0  # To normalize the Gaussian weighting
        
        for n in np.arange(-4 * sigma, 4 * sigma, step):
            shifted_index = i - int(n / step)  # Find the shifted index corresponding to (x - n)
            if 0 <= shifted_index < len(unsmoothed_curve):
                PCT_value = unsmoothed_curve[shifted_index]
            else:
                PCT_value = 0  # Out of bounds: assume power curve is 0
            
            weight = gaussian_filter(n, sigma)
            aggregate_power += PCT_value * weight
            normalization_factor += weight  # Sum of weights for normalization

        smoothed_curve[i] = aggregate_power / normalization_factor  # Normalize the sum by weights

    return smoothed_curve

In [79]:
# Apply smoothing with different smoothing levels
smoothing_level = 0.20
smoothed_power_curve = smooth_power_curve(power_values_interp, wind_speeds_interp, smoothing_level)

### Step 4: Calculate wind generation and CF

In [59]:
turbine = {'hub_height':100, 'P':15000., 'V':wind_speeds_interp, 'POW':smoothed_power_curve} # we assign the turbine at 100m because we have to trick atlite into thinking the wind is at 100 m to avoid an error

In [60]:
layout = cutout_ie.layout_from_capacity_list(df_capacity_offshore)
wind = cutout_ie.wind(turbine=turbine, layout=layout, add_cutout_windspeed=True)

In [66]:
wind_cf = wind[0]/layout.sum()

In [78]:
wind_cf.to_netcdf('../data/wind_offshore_cf_1940_2023.nc')